In [1]:
type NonlinearFunctionParameters{TT,FT,D,S}
    f::FT
    Δt::TT
end

In [2]:
type NonlinearFunctionCache{DT}
    q::Vector{DT}
    f::Vector{DT}
end

In [3]:
d = 2;
s = 2;

In [4]:
Δt = 0.1;

In [5]:
function pendulum_ode_f(x, f)
    f[1] = x[2]
    f[2] = sin(x[1])
    nothing
end

pendulum_ode_f (generic function with 1 method)

In [6]:
params = NonlinearFunctionParameters{typeof(Δt), typeof(pendulum_ode_f), d, s}(pendulum_ode_f, Δt)

NonlinearFunctionParameters{Float64,#pendulum_ode_f,2,2}(pendulum_ode_f,0.1)

In [7]:
@generated function function_stages!{DT,TT,FT,D,S}(x::Vector{DT}, b::Vector{DT}, params::NonlinearFunctionParameters{TT,FT,D,S})
    sol_q = zeros(DT, D)
    sol_f = zeros(DT, D)
    
    cache = NonlinearFunctionCache{DT}(sol_q, sol_f)
    
    function_stages = quote
        @assert length(x) == length(b)
        
        $cache.q .= x
        params.f($cache.q, $cache.f)
        b .= $cache.f
    end
    
    return function_stages
end

function_stages! (generic function with 1 method)

In [8]:
x1 = zeros(Float64, d);
b1 = zeros(Float64, d);

In [9]:
@time function_stages!(x1, b1, params)

  0.070683 seconds (20.44 k allocations: 1.172 MB)


2-element Array{Float64,1}:
 0.0
 0.0

In [10]:
@time function_stages!(x1, b1, params)

  0.000004 seconds (4 allocations: 160 bytes)


2-element Array{Float64,1}:
 0.0
 0.0

In [11]:
x2 = zeros(Int64, d);
b2 = zeros(Int64, d);

In [12]:
@time function_stages!(x2, b2, params)

  0.021753 seconds (6.55 k allocations: 300.558 KB)


2-element Array{Int64,1}:
 0
 0

In [13]:
@time function_stages!(x2, b2, params)

  0.000004 seconds (4 allocations: 160 bytes)


2-element Array{Int64,1}:
 0
 0

In [17]:
function_stages = (x,b) -> function_stages!(x, b, params)

(::#7) (generic function with 1 method)

In [22]:
@time function_stages(x1, b1)

  0.003610 seconds (195 allocations: 11.531 KB)


2-element Array{Float64,1}:
 0.0
 0.0

In [23]:
@time function_stages(x1, b1)

  0.000005 seconds (4 allocations: 160 bytes)


2-element Array{Float64,1}:
 0.0
 0.0

In [20]:
@time function_stages(x2, b2)

  0.003410 seconds (195 allocations: 11.531 KB)


2-element Array{Int64,1}:
 0
 0

In [21]:
@time function_stages(x2, b2)

  0.000005 seconds (4 allocations: 160 bytes)


2-element Array{Int64,1}:
 0
 0